In [ ]:
import kagglehub
import shutil
import os

# Download dataset (will be saved in the default location)
path = kagglehub.dataset_download("thienkhonghoc/affectnet")

# Define your target directory
target_dir = "/content/testdata"

# Ensure the target directory exists
os.makedirs(target_dir, exist_ok=True)

# Move the downloaded dataset to the target directory
shutil.move(path, target_dir)

print("Dataset successfully moved to:", target_dir)

100%|██████████| 1.75G/1.75G [01:20<00:00, 23.3MB/s]

Extracting files...


Dataset successfully moved to: /content/testdata


In [ ]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
TEST_DIR = '/content/testdata/3/AffectNet/test'
TRAIN_DIR='/content/testdata/3/AffectNet/train'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        if label =='contempt' or label =='labels.csv':
            continue
        for imagename in os.listdir(os.path.join(dir,label)):

            image_paths.append(os.path.join(dir,label,imagename))

            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

6 completed
7 completed
2 completed
5 completed
3 completed
4 completed
0 completed
1 completed


In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

6 completed
7 completed
2 completed
5 completed
3 completed
4 completed
0 completed
1 completed


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):

        img = load_img(image, color_mode="rgb", target_size=(96, 96))
        img = np.array(img)
        features.append(img)
    features = np.array(features)

    features = features.reshape(len(features), 96, 96, 3)
    return features

In [ ]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])
x_train = train_features/255.0
x_test = test_features/255.0

  0%|          | 0/37553 [00:00<?, ?it/s]

  0%|          | 0/3200 [00:00<?, ?it/s]

In [ ]:
print("Unique labels in train data:", train['label'].unique())
print("Unique labels in test data:", test['label'].unique())

Unique labels in train data: ['6' '7' '2' '5' '3' '4' '0' '1']
Unique labels in test data: ['6' '7' '2' '5' '3' '4' '0' '1']


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the label encoder
le = LabelEncoder()
le.fit(train['label'])

# Print the mapping
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

Label mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7}


In [ ]:
from sklearn.preprocessing import LabelEncoder
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))

Label mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7}


In [ ]:
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train,num_classes = 8)
y_test = to_categorical(y_test,num_classes = 8)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',  input_shape=(96, 96, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

# Changed final layer to output 8 classes instead of 7
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
print("Training data shape:", x_train.shape)
print("Training labels shape:", y_train.shape)
print("Model output shape:", model.output_shape)

Training data shape: (37553, 96, 96, 3)
Training labels shape: (37553, 8)
Model output shape: (None, 8)


In [ ]:
hist = model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 49s 97ms/step - accuracy: 0.1428 - loss: 2.2406 - val_accuracy: 0.1322 - val_loss: 2.0815
Epoch 2/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.1607 - loss: 2.0595 - val_accuracy: 0.1847 - val_loss: 1.9974
Epoch 3/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.2522 - loss: 1.9087 - val_accuracy: 0.2869 - val_loss: 1.8157
Epoch 4/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.3251 - loss: 1.7390 - val_accuracy: 0.3200 - val_loss: 1.7452
Epoch 5/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.3673 - loss: 1.6447 - val_accuracy: 0.3688 - val_loss: 1.6307
Epoch 6/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.4144 - loss: 1.5488 - val_accuracy: 0.4106 - val_loss: 1.5412
Epoch 7/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.4357 - loss: 1.4914 - val_accuracy: 0.3913 - val_loss: 1.5448
Epoch 8/100
294/294 ━━━━━━━━━━━━━━━━━━━━ 9s 32ms/step - accuracy: 0.4603 - loss: 1.4281 

In [ ]:
model.save("/content/Models/emotiondetector.h5")